<h1> Battle of the Neighborhoods (Austin Food Tour): </h1>

<h3> Outline:</h3>
<p> 1. get data on austin neighborhoods and their locations</p>
<p> 2. get foursquare data on the most frequent restaurant types in each neighborhood</p>
<p> cluster the neighborhoods by types of restaurants</p>
<p> within each cluster, identify the top 5 restaurants for the top restaurant type and then write them down in a list</p>

<h2> Introduction/Business Problem:</h2>

<p style="font-size:16px">In this notebook, we want to cluster neighborhoods in Austin based on the cuisine of the restaurants they offer. Then, we want to find the top 5 restaurants in each cluster (according to rating) to create a list of must-go restaurants. The stake holders are the tourism board of Austin (to drive heavier tourist traffic around every part of Austin), travel bloggers, tourists, and college students in the area. </p>

<h2>Data</h2>

<p style="font-size:16px"> We need data on the neighborhoods of Austin and their locations (using Wikipedia and Nominatim) and data on the most frequent kind of restaurant and the most popular restaurants in each neighborhood (from Foursquare). </p>